# Configuration Variable Coverage Diagnostics

## Purpose

This notebook evaluates which configuration-related variables have sufficient empirical support to define **within-size structural variation** in residential solar systems.

The goal is **not** to analyze structure yet.

The goal is to determine, empirically and transparently:
- which variables are populated,
- over what time span,
- and whether they can plausibly support configuration analysis.

This step satisfies the requirement to **record and report how exclusion decisions are made** before restricting the analytical space.
